<a href="https://colab.research.google.com/github/dolDolSee/TF2.0/blob/main/albumentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!wget https://www.sciencenews.org/wp-content/uploads/2020/03/033120_HT_covid-cat_feat-1028x579.jpg
!ls -lia

In [ ]:
import cv2
import matplotlib.pyplot as plt

org_image = cv2.cvtColor(cv2.imread('033120_HT_covid-cat_feat-1028x579.jpg'),cv2.COLOR_BGR2RGB)
def show_image(image):
    plt.figure(figsize=(8,8))
    plt.imshow(image)
    plt.axis('off')
show_image(org_image)


In [ ]:
import albumentations as A

augmentor = A.HorizontalFlip(p=0.5)
#딕셔너리 형태로 들어옴
aug_image = augmentor(image=org_image)['image']
show_image(aug_image)

In [ ]:
def show_images(images, labels, ncols=4, title=None):
    figure, axs = plt.subplots(figsize=(22, 4), nrows=1, ncols=ncols)
    for i in range(ncols):
        axs[i].imshow(images[i])
        axs[i].set_title(labels[i])

# augmentor로 입력된 albumentations augmentation을 반복 수행
def repeat_aug(count=4, org_image=None, label=None, augmentor=None):
    image_list = [org_image]
    label_list = ['original']
    
    for i in range(count):
        aug_image = augmentor(image=org_image)['image']
        image_list.append(aug_image)
        label_list.append(label)
              
    show_images(image_list, label_list, ncols=count+1)
    
# 90도 범위안에서 돌아가면서 빈 영역은 검은색으로 채우겠다.
augmentor = A.Rotate(limit=90, p=1, border_mode=cv2.BORDER_CONSTANT)
#aug_image = augmentor(image=org_image)['image']

repeat_aug(count=4, org_image=org_image, label='Rotate', augmentor=augmentor)

In [ ]:
import albumentations as A

augmentor = A.HorizontalFlip(p=0.5)
aug_image = augmentor(image=org_image)['image']

repeat_aug(count=4, org_image=org_image, label='HorizontalFlip', augmentor=augmentor)

In [ ]:
import albumentations as A

augmentor = A.VerticalFlip(p=0.5)
repeat_aug(count=4, org_image=org_image, label='VerticalFlip', augmentor=augmentor)

In [ ]:
augmentor = A.Rotate(limit=90, p=1, border_mode = cv2.BORDER_REFLECT)
repeat_aug(count=4, org_image=org_image, label="Rotate",augmentor=augmentor)

In [ ]:
augmentor = A.RandomRotate90(p=1)
repeat_aug(count=4, org_image=org_image, label='RandomRotate90', augmentor=augmentor)

In [ ]:
augmentor = A.ShiftScaleRotate(shift_limit=0, scale_limit=(0.5,1.0),rotate_limit=0,p=1)
repeat_aug(count=3, org_image=org_image, label='only_scale', augmentor=augmentor)

In [ ]:
# 이동 후 남게 되는 border를 검은색으로 fill
augmentor = A.ShiftScaleRotate (shift_limit=(0.2, 0.1), scale_limit=0, 
                                rotate_limit=0, border_mode=cv2.BORDER_CONSTANT, p=1)
repeat_aug(count=3, org_image=org_image, label='only_shift_fillblack', augmentor=augmentor)


In [ ]:
# 이동 후 남게 되는 Border를 Near 영역이미지로 복사
augmentor = A.ShiftScaleRotate (shift_limit=(0.2, 0.1), scale_limit=0, 
                                rotate_limit=0, p=1)
repeat_aug(count=3, org_image=org_image, label='only_shift_fillcopy', augmentor=augmentor)

In [ ]:
# Shift, Scale, Rotate를 한꺼번에 적용. 
augmentor = A.ShiftScaleRotate (p=1)
repeat_aug(count=4, org_image=org_image, label='ShiftScaleRotate', augmentor=augmentor)

In [ ]:
augmentor = A.Compose([
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5)
])
repeat_aug(count=6, org_image=org_image, label='Composite', augmentor=augmentor)
repeat_aug(count=6, org_image=org_image, label='Composite', augmentor=augmentor)

In [ ]:
# Crop은 원본 이미지의 특정 영역을 잘라낸 후, 원본 사이즈로 다시 Resize하지 않으므로 유의 필요. 
augmentor = A.Crop(x_min=100, y_min=100, x_max=500, y_max=500, p=1)
aug_image = augmentor(image=org_image)['image']
print('crop image size:', aug_image.shape)
repeat_aug(count=4, org_image=org_image, label='Crop', augmentor=augmentor)

# Crop 적용 후 반드시 아래와 같이 resize 해야 함.(augmentation이후 opencv resize()등을 적용해도 무관)
augmentor = A.Compose([
    A.Crop(x_min=100, y_min=100, x_max=500, y_max=500, p=1),
    A.Resize(578, 1028)
])
aug_image = augmentor(image=org_image)['image']
print('crop and resize image size:', aug_image.shape)
repeat_aug(count=4, org_image=org_image, label='Crop', augmentor=augmentor)


In [ ]:
# CenterCrop은 원본 이미지의 중심을 기준으로 입력된 height, width 값 만큼의 영역을 Crop 한 뒤 원본 이미지 사이즈로 Resize하지 않음. 
augmentor = A.CenterCrop(height=300, width=500, p=1)
aug_image = augmentor(image=org_image)['image']

repeat_aug(count=4, org_image=org_image, label='CenterCrop', augmentor=augmentor)
print('centercrop image size:', aug_image.shape)

# CenterCrop 후에는 반드시 아래와 같이 resize 해야 함.(augmentation이후 opencv resize()등을 적용해도 무관)
augmentor = A.Compose([
    A.CenterCrop(height=300, width=500, p=1),
    A.Resize(578, 1028)
])
aug_image = augmentor(image=org_image)['image']
print('centercrop and resize image size:', aug_image.shape)
repeat_aug(count=4, org_image=org_image, label='CenterCrop_resize', augmentor=augmentor)

In [ ]:
# RandomCrop은 height, width 값 만큼의 영역을 Random하게 Crop 한 뒤 원본 이미지 사이즈로 Resize하지 않음. 
augmentor = A.RandomCrop(height=400, width=800, p=1)
aug_image = augmentor(image=org_image)['image']

repeat_aug(count=4, org_image=org_image, label='random_crop', augmentor=augmentor)
print('centercrop image size:', aug_image.shape)

# RandomCrop 후에는 반드시 아래와 같이 resize 해야 함.(augmentation이후 opencv resize()등을 적용해도 무관)
augmentor = A.Compose([
    A.RandomCrop(height=400, width=800, p=1),
    A.Resize(578, 1028)
])
aug_image = augmentor(image=org_image)['image']
print('centercrop and resize image size:', aug_image.shape)
repeat_aug(count=4, org_image=org_image, label='random_crop', augmentor=augmentor)

In [ ]:
augmentor = A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=1)
repeat_aug(count=4, org_image=org_image, label='bright_contrast', augmentor=augmentor)

augmentor = A.RandomBrightnessContrast(brightness_limit=(0, 0.2), contrast_limit=(0, 0.2), p=1)
repeat_aug(count=4, org_image=org_image, label='bright_contrast', augmentor=augmentor)

# 밝기는 고정하고 대비만 변경. 
augmentor = A.RandomBrightnessContrast(brightness_limit=0, contrast_limit=(0, 0.2), p=1)
repeat_aug(count=4, org_image=org_image, label='bright_contrast', augmentor=augmentor)

In [ ]:
import albumentations as A

augmentor = A.HueSaturationValue(p=1)
repeat_aug(count=4, org_image=org_image, label='HueSaturationValue', augmentor=augmentor)



In [ ]:
augmentor = A.RGBShift(p=1)
repeat_aug(count=4, org_image=org_image, label='RGBShift', augmentor=augmentor)

In [ ]:
augmentor = A.ChannelShuffle(p=1)
repeat_aug(count=4, org_image=org_image, label='RGBShift', augmentor=augmentor)

In [ ]:
augmentor_hsv = A.HueSaturationValue(p=1)
augmentor_rgb = A.RGBShift(p=1)
augmentor_channel = A.ChannelShuffle(p=1)
repeat_aug(count=4, org_image=org_image, label='HSV', augmentor=augmentor_hsv)
repeat_aug(count=4, org_image=org_image, label='RGBShift', augmentor=augmentor_rgb)
repeat_aug(count=4, org_image=org_image, label='ChannelShuffle', augmentor=augmentor_channel)

In [ ]:
augmentor_hsv = A.ColorJitter(p=1)
repeat_aug(count=4, org_image=org_image, label='ColorJitter', augmentor=augmentor_hsv)

In [ ]:
augmentor = A.GaussNoise(p=1, var_limit=(100, 200))
repeat_aug(count=2, org_image=org_image, label='GaussNoise', augmentor=augmentor)

In [ ]:
augmentor = A.Cutout(p=1)
repeat_aug(count=2, org_image=org_image, label='Cutout', augmentor=augmentor)

augmentor = A.CoarseDropout(p=1, max_holes=26)
repeat_aug(count=2, org_image=org_image, label='CoarseDropout', augmentor=augmentor)

In [ ]:
augmentor_hsv = A.CLAHE(p=1, clip_limit=4)
repeat_aug(count=2, org_image=org_image, label='CLAHE', augmentor=augmentor_hsv)

In [ ]:
augmentor = A.Blur(p=1, blur_limit=(7, 10))
repeat_aug(count=4, org_image=org_image, label='Blur', augmentor=augmentor)

# GaussianBlur의 blur_limit 즉 kernel size는 홀수가 되어야 함. 따라서 blur_limit=(3, 8)과 같이 짝수를 입력하면 안됨. 
augmentor = A.GaussianBlur(p=1, blur_limit=(11, 15))
repeat_aug(count=4, org_image=org_image, label='GaussBlur', augmentor=augmentor)

In [ ]:
augmentor = A.RandomRain(p=1)
repeat_aug(count=2, org_image=org_image, label='Rain', augmentor=augmentor)

augmentor = A.RandomShadow(p=1)
repeat_aug(count=2, org_image=org_image, label='Shadow', augmentor=augmentor)

augmentor = A.RandomSnow(p=1)
repeat_aug(count=2, org_image=org_image, label='Snow', augmentor=augmentor)

In [ ]:
augmentor = A.Compose([
    A.CenterCrop(height=300, width=800),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Resize(579,1024,p=1)],p=0.5)
repeat_aug(count=4, org_image=org_image, label='Composite', augmentor=augmentor)

In [ ]:
# OneOf() 내에서 VerticalFlip, HorizontalFlip, RandomBrightnessContrast를 각각을 p=1 로 부여했지만 연속해서 Augmentation이 적용 되지 않고, 개별 변환만 적용됨
augmentor = A.OneOf([
    A.VerticalFlip(p=1),
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(45, 90), p=1, border_mode=cv2.BORDER_CONSTANT)
    ], p=1)
repeat_aug(count=4, org_image=org_image, label='OneOf', augmentor=augmentor)

In [ ]:
augmentor = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.OneOf([
        A.CLAHE(p=0.3),
        A.Blur(blur_limit=(10, 15), p=0.3)
    ], p=0.5)
])
repeat_aug(count=4, org_image=org_image, label='OneOf', augmentor=augmentor)
repeat_aug(count=4, org_image=org_image, label='OneOf', augmentor=augmentor)